Election de 2022 premier tour

In [1]:
import numpy as np
import pandas as pd
import os
os.listdir()

['migrations',
 'election2017.ipynb',
 '.DS_Store',
 'alembic.ini',
 'economie_chomage.ipynb',
 'economie',
 'menage_epargne.ipynb',
 'enums',
 'utils',
 'models',
 'election2012.ipynb',
 'exports',
 'labo1.ipynb',
 '.gitignore',
 'population.ipynb',
 'scripts',
 'db',
 'securite.ipynb',
 'chomages',
 'labo2.ipynb',
 'population_immigration.ipynb',
 '.git',
 'main.py',
 'election2022.ipynb',
 'data',
 'menage_prix_consomation.ipynb',
 'menage_depense.ipynb']

#### Importer la database

Import des datasets.

In [13]:
df_prix_cosomation = pd.read_excel('./data/economie/menage/prix-consommation-base-2018 .xlsx')
df_depense = pd.read_excel('./data/economie/menage/depenses.xlsx')
df_epargne = pd.read_excel('./data/economie/menage/epargne.xlsx')

Visualise toutes les colones

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

df_prix_cosomation.head()

,Année,Taux d'épargne
0,1960,17.0
1,1961,16.2
2,1962,18.3
3,1963,17.8
4,1964,17.6


In [ ]:
df_depense.head()

,Année,Dépenses pré-engagées,Dépenses liées au logement,Services de télévision et de télécommunications
0,1960,12.6,9.5,0.5
1,1961,13.1,10.0,0.5
2,1962,13.2,10.2,0.5
3,1963,13.9,10.7,0.5
4,1964,14.2,10.9,0.5


In [17]:
df_epargne.head()

,Année,Taux d'épargne
0,1960,17.0
1,1961,16.2
2,1962,18.3
3,1963,17.8
4,1964,17.6


Renommer les colonnes 

In [19]:
df_prix_cosomation.columns = [
    "annee", "prix_consommation"
]

df_epargne.columns = [
    "annee", "taux_epargne"
]

df_depense.columns = [
    "annee", "pre_engagees", "logement", "service_multimedia",
]

Normalisation structurelle

In [36]:
def normaliser_df_contexte(
    df: pd.DataFrame,
    colonnes_numeriques: list[str],
    colonne_annee: str = "annee"
) -> pd.DataFrame:
    """
    Normalise un DataFrame de données de contexte annuel.

    - Extrait l'année sur 4 chiffres
    - Nettoie les annotations (ex: (r), %, virgules)
    - Convertit les colonnes numériques en float

    :param df: DataFrame source
    :param colonnes_numeriques: Liste des colonnes à convertir en float
    :param colonne_annee: Nom de la colonne année (par défaut 'annee')
    :return: DataFrame normalisé
    """

    df = df.copy()

    # ============================
    # Colonnes numériques
    # → annee en premier, sans doublon
    # ============================

    colonnes_ordre = [
        colonne_annee,
        *[col for col in colonnes_numeriques if col != colonne_annee]
    ]
    autres_colonnes = [c for c in df.columns if c not in colonnes_ordre]
    df = df[colonnes_ordre + autres_colonnes]


    # ============================
    # Normalisation de l'année
    # ============================
    df[colonne_annee] = (
        df[colonne_annee]
        .astype(str)
        .str.extract(r"(\d{4})")[0]
        .astype(int)
    )

    # ============================
    # Normalisation des colonnes numériques
    # ============================
    for col in colonnes_numeriques:
        df[col] = (
            df[col]
            .astype(str)
            # Supprimer les annotations type (r), (p), (e)…
            .str.replace(r"^\(.*?\)\s*", "", regex=True)
            # Supprimer les %
            .str.replace("%", "", regex=False)
            # Virgule → point
            .str.replace(",", ".", regex=False)
            .str.strip()
            # Valeurs non convertibles → NaN
            .astype(float)
        )

    return df[(df["annee"] >= 2007) & (df["annee"] <= 2022)]


In [40]:
from functools import reduce
import pandas as pd

def fusionner_contexte_menage(
    dfs: list[pd.DataFrame],
    colonne_annee: str = "annee"
) -> pd.DataFrame:
    """
    Fusionne plusieurs DataFrames de contexte ménage sur l'année.

    - Jointure OUTER pour ne perdre aucune année
    - Une ligne par année
    """

    # Sécurité : garder uniquement les DF contenant la colonne année
    dfs_valides = [
        df for df in dfs if colonne_annee in df.columns
    ]

    if not dfs_valides:
        raise ValueError("Aucun DataFrame ne contient la colonne 'annee'")

    df_fusion = reduce(
        lambda left, right: pd.merge(
            left,
            right,
            on=colonne_annee,
            how="outer"
        ),
        dfs_valides
    )

    return df_fusion.sort_values(colonne_annee).reset_index(drop=True)


In [37]:
df_prix_cosomation = normaliser_df_contexte(df_prix_cosomation, colonnes_numeriques=["prix_consommation"])
df_epargne = normaliser_df_contexte(df_epargne, colonnes_numeriques=["taux_epargne"])
df_depense = normaliser_df_contexte(df_depense, colonnes_numeriques=["pre_engagees", "logement", "service_multimedia"])


In [41]:
df = fusionner_contexte_menage([
    df_prix_cosomation,
    df_epargne,
    df_depense
])


In [44]:
df.head(16)

,annee,prix_consommation,taux_epargne,pre_engagees,logement,service_multimedia
0,2007,1.5,14.8,28.4,20.9,2.8
1,2008,2.8,14.8,29.0,21.2,2.9
2,2009,0.1,16.1,28.9,21.5,2.9
3,2010,1.5,16.0,28.7,21.7,2.8
4,2011,2.1,15.6,28.6,21.8,2.7
5,2012,2.0,15.6,29.0,22.5,2.5
6,2013,0.9,14.2,29.8,23.2,2.4
7,2014,0.5,14.6,29.4,23.0,2.3
8,2015,0.0,14.1,29.6,23.1,2.2
9,2016,0.2,13.9,29.4,23.0,2.2


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   annee               16 non-null     int64  
 1   prix_consommation   16 non-null     float64
 2   taux_epargne        16 non-null     float64
 3   pre_engagees        16 non-null     float64
 4   logement            16 non-null     float64
 5   service_multimedia  16 non-null     float64
dtypes: float64(5), int64(1)
memory usage: 900.0 bytes
